# Assignment 3 Group no. []
### Project members: 



### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [2]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def accuracy(df, correctlabels):
    N = range(len(df))
    pred = [df.iloc[i,:].idxmax() for i in N]
    indicator = [1 if pred[i] == correctlabels[i] else 0 for i in N ]
    accuracy = sum(indicator)/len(indicator)
    return accuracy

def brier_score(df, correctlabels):
    if len(df) != len(correctlabels):
        print("Error in Length")
        return None
    sum_error = 0
    for i in range(len(df)):
        f = df.iloc[i,:]
        idx = np.where(df.columns==correctlabels[i])[0]
        o = [1 if k == idx else 0 for k in range(df.shape[1])]
        sum_error += np.sum((f - o)**2)
    
    return sum_error / len(df)

def binary_auc(pred, true, col):
    
    pred = np.array(pred)
    v = [i == col for i in true] 
    col_pred, other_pred = pred[v], pred[[not i for i in v]]

    predscore = list(pred)
    predscore.extend([0,1])
    predscore = sorted(predscore)
    dictscores = {}

    for thres in predscore:
        if(len(other_pred)==0):
            FPR = 0
        else:
            FPR = sum(other_pred >= thres)/len(other_pred)
        if(len(col_pred)==0):
            TPR = 0
        else:
            TPR = sum(col_pred >= thres)/len(col_pred)
        dictscores[thres] = [FPR, TPR]

    list_reversed = [i for i in reversed(list(dictscores.values()))]

    auc = 0
    if([0,0] not in list_reversed):
        list_reversed = [[0,0]] + list_reversed
    if([1,1] not in list_reversed):
        list_reversed = list_reversed + [[1,1]]
    for i in range(len(list_reversed) - 1):
        tpr_left, tpr_right = list_reversed[i][1], list_reversed[i+1][1]
        fpr_left, fpr_right = list_reversed[i][0], list_reversed[i+1][0]
        if fpr_right != fpr_left:
            height = (tpr_left + tpr_right) / 2
            width = fpr_right - fpr_left
            auc += height*width
    return auc


def auc(df, correctlabels):
    N = len(df)
    auc = 0
    for col in df:
        weight = sum(pd.Series(correctlabels) == col)/N
        bin_auc = binary_auc(df[col], correctlabels, col)
        auc += weight*bin_auc
    return auc

def create_normalization(df, normalizationtype="minmax"):
    new_df = df.copy()
    normalization = {}
    for col in df.columns:
        if col == "CLASS" or col == "ID":
            continue
        if normalizationtype == "minmax":
            a = df[col].min()
            b = df[col].max()
            new_df[col] = (df[col] - a)/(b - a)
            normalization.update({col: (normalizationtype, a, b)})
        elif normalizationtype == "zscore":
            a = df[col].mean()
            b = df[col].std()
            new_df[col] = (df[col] - a)/b
            normalization.update({col: (normalizationtype, a, b)})
        else:
            print("There is no such type!")
        
    return new_df, normalization

def apply_normalization(df, normalization):
    new_df = df.copy()
    for col in normalization:
        normalizationtype = normalization[col][0]
        a = normalization[col][1]
        b = normalization[col][2]
        if normalizationtype == "minmax":
            new_df[col] = (df[col] - a)/(b - a)
            new_df[col] = [x if x < 1 else 1 for x in new_df[col] ] #hint 2
            new_df[col] = [x if x > 0 else 0 for x in new_df[col] ] #hint 2
        elif normalizationtype == "zscore":
            new_df[col] = (df[col] - a)/b
        else:
            print("There is no such type!")
    
    return new_df

def create_imputation(df):
    new_df = df.copy()
    imputation = {}

    for col in df.columns:
        if col not in ["ID", "CLASS"]:
            if df[col].dtype in ['float64', 'float32', 'int64', 'int32']: 
                fillvalue = df[col].mean()
                new_df[col] = new_df[col].fillna(fillvalue)
            elif df[col].dtype == "object" or df[col].dtype.name == "category": 
                fillvalue = df[col].mode()[0]
                new_df[col] = new_df[col].fillna(fillvalue)       
            imputation.update({col: fillvalue})
    return new_df, imputation

def apply_imputation(df, imputation):
    new_df = df.copy()
    for col in df.columns:
        if col not in ['ID','CLASS']:     
            fillvalue = imputation[col]
            new_df[col] = new_df[col].fillna(fillvalue)
    return new_df

def create_bins(df, nobins=10, bintype="equal-width"):
    new_df = df.copy()
    binning = {}
    for col in new_df.columns:
        if col not in ['ID', 'CLASS'] and new_df[col].dtype in ['float64', 'float32', 'int64', 'int32']:
            if bintype == 'equal-width':
                new_df[col], bins = pd.cut(new_df[col], nobins, retbins=True, labels=False)
            elif bintype == 'equal-size':
                new_df[col], bins = pd.qcut(new_df[col], nobins, retbins=True, labels=False, duplicates='drop')
            bins[0], bins[-1] = -np.inf, np.inf
            binning[col] = bins
            new_df[col] = new_df[col].astype('category')
            new_df[col] = new_df[col].cat.set_categories([str(i) for i in new_df[col].cat.categories], rename=True) #int->str
        else:
            new_df[col] = new_df[col].astype('category')
    return new_df, binning


def apply_bins(df, binning):
    new_df = df.copy()
    for col in new_df.columns:
        if(col not in ['ID', 'CLASS'] and new_df[col].dtype in ['float64', 'float32', 'int64', 'int32']):
            bins = binning[col]
            new_df[col] = pd.cut(new_df[col], bins, labels=False)
            new_df[col] = new_df[col].astype('category')
            new_df[col] = new_df[col].cat.set_categories([str(i) for i in new_df[col].cat.categories], rename = True) 
        else:
            new_df[col] = new_df[col].astype('category')
    return new_df

def create_one_hot(df):
    new_df = df.copy()
    one_hot = {}
    for col in df.columns:
        if col not in ['ID', 'CLASS']: 
            if new_df[col].dtype == 'object' or new_df[col].dtype.name ==  'category':
                new_df[col] = new_df[col].astype('category')
                one_hot[col] = new_df[col].cat.categories    
                df_one_hot = pd.get_dummies(new_df[col])
                new_df = new_df.drop(columns=col, axis=1)
                new_df = pd.concat([new_df, df_one_hot], axis=1)
                
    return new_df, one_hot

def apply_one_hot(df, one_hot):
    new_df = df.copy()
    for col in df.columns:
        if col in one_hot.keys():
            for i in one_hot[col]:
                name = col+'_'+i
                new_col = pd.Series((df[col]==i).astype('float'))
                new_df[name] = new_col
            new_df = new_df.drop(columns=col, axis=1)
    return new_df




## 1. Define the class DecisionTree

In [3]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
#
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
#
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
#
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
#
# Hint 5: The best feature found (with lowest residual information content) will be used to split the training
#         instances, and each sub-group will be used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
#
# Hint 6: You may make divide_and_conquer return not only a list of nodes, but also a current_node_no; 
#         by this, each subsequent call to divide_and_conquer for each subset of instances, i.e. for each feature value, 
#         could use current_node_no as a starting point.
#         If you e.g. make the following call:
#
#         current_node_no, node_list = divide_and_conquer(current_node_no, ...)
#
#         then the returned value in current_node_no can be used in the next call to divide_and_conquer.
#         Node that node_list will contain an arbitrary number of tuples, each element corresponding to a node together 
#         with a node number. The first element in the list will have the same number as current_node_no when the call 
#         was made and the last element will have a number one less than current_node_no when returned, e.g., if there is
#         only one (leaf) node in the returned list, then current_node_no will only be incremented by one through the above call.
#         
# Hint 7: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root



In [6]:
class DecisionTree:
    
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None

        
    def information_content(self, group_size, class_counts):
        group_weight = {k:group_size[k]/sum(group_size.values())  for k in group_size}
        InfRes = 0 
        for key in group_weight:
            tmp = {k:class_counts[k]  for k in class_counts if k[0] == key}
            tmp = {k:tmp[k]/sum(tmp.values())  for k in tmp}
            LnP = np.sum([-tmp[k]*np.log2(tmp[k]) for k in tmp])
            InfRes += group_weight[key]*LnP
        return InfRes        
    
    def divide_and_conquer(self, df, min_samples_split, nodeno):
        if nodeno == 0:
            self.max_nodes_no = 0
            self.model = []
        best_idx = 0
        best_info = np.inf
        
        features = [x for x in df.columns if x != "CLASS"]
        for i in range(len(features)):
            ft = features[i]
            group_size = df[ft].value_counts().to_dict()  
            class_counts = df.groupby([ft, "CLASS"]).size().to_dict() 
            info = self.information_content(group_size, class_counts)
            if info < best_info:
                best_info = info
                best_idx = i
        best_feature = features[best_idx]

        possiblevalues = df[best_feature].unique()

        node_dict = {}
        df.set_index(best_feature, inplace=True)
        for pv in possiblevalues:
            
            self.max_nodes_no += 1
            node_dict[pv] = self.max_nodes_no
            
            new_df = df.loc[[pv]]
            if new_df.shape[0] <= min_samples_split or len(features) == 1 :
                self.model.append((self.max_nodes_no, "leaf", new_df.CLASS.value_counts(normalize=True)))
            else:
                self.model.append((self.divide_and_conquer(new_df, min_samples_split, self.max_nodes_no)))
        res = (nodeno, best_feature, node_dict)
        return res
 
    def make_prediction(self, nodeno, row):
        nodeno, nodevalue, dic = self.model[nodeno] 
        if nodevalue == "leaf":
            return dic.sort_index()
        else:
            row_feature_value = row[nodevalue]
            if row_feature_value in dic.keys():
                return self.make_prediction(dic[row_feature_value], row) 
            else:
                # exception
                possiblenodes = list(dic.values())
                res = np.zeros([len(possiblenodes), len(self.labels)])
                for i in range(res.shape[0]):
                     res[i] = self.make_prediction(possiblenodes[i], row)
                res = np.mean(res, axis = 0)
                return res
    
    
    def fit(self, df, nobins=10, bintype='equal-width', min_samples_split=5):
        self.training_data = df.copy()
        self.training_labels = df["CLASS"].astype("category")
        self.labels = self.training_labels.cat.categories    
        self.count = self.training_labels.value_counts()  
        freq = self.count/sum(self.count)
        self.class_priors = freq.to_dict() # default class probabilities (relative frequencies of the class labels)
        del self.training_data["ID"]
        
        self.training_data, self.imputation = create_imputation(self.training_data)
        self.training_data, self.binning = create_bins(self.training_data, nobins=nobins, bintype=bintype)

        tree_model = self.divide_and_conquer(self.training_data, min_samples_split, 0)
        self.model.append(tree_model)
        self.model = sorted(self.model, key=lambda x:x[0])
        
            
    def predict(self, df):
        self.testing_data = df.copy()
  
        if "CLASS" in self.testing_data.columns:
            del self.testing_data["CLASS"]
        if "ID" in self.testing_data.columns:
            del self.testing_data["ID"]
        
        self.testing_data = apply_imputation(self.testing_data, self.imputation)
        self.testing_data = apply_bins(self.testing_data, self.binning)

        predictions = np.array([np.zeros(len(self.labels)) for i in range(self.testing_data.shape[0])])
        for i in range(self.testing_data.shape[0]):
            row = self.testing_data.iloc[i,:]
            predictions[i] = self.make_prediction(0, row)

        predictions = pd.DataFrame(predictions, columns=self.labels)
        
        return predictions


In [7]:
# Test our code 

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 0.78 s.
Testing time (5, 'equal-width', 3): 0.30 s.
Training time (5, 'equal-width', 5): 0.77 s.
Testing time (5, 'equal-width', 5): 0.16 s.
Training time (5, 'equal-width', 10): 0.33 s.
Testing time (5, 'equal-width', 10): 0.12 s.
Training time (5, 'equal-size', 3): 0.64 s.
Testing time (5, 'equal-size', 3): 0.12 s.
Training time (5, 'equal-size', 5): 0.36 s.
Testing time (5, 'equal-size', 5): 0.10 s.
Training time (5, 'equal-size', 10): 0.26 s.
Testing time (5, 'equal-size', 10): 0.10 s.
Training time (10, 'equal-width', 3): 0.57 s.
Testing time (10, 'equal-width', 3): 0.12 s.
Training time (10, 'equal-width', 5): 0.54 s.
Testing time (10, 'equal-width', 5): 0.11 s.
Training time (10, 'equal-width', 10): 0.31 s.
Testing time (10, 'equal-width', 10): 0.11 s.
Training time (10, 'equal-size', 3): 0.50 s.
Testing time (10, 'equal-size', 3): 0.16 s.
Training time (10, 'equal-size', 5): 0.52 s.
Testing time (10, 'equal-size', 5): 0.18 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.635514     0.569085  0.787104
               5   0.607477     0.573103  0.789411
               10  0.570093     0.565079  0.791692
   equal-size  3   0.598131     0.689916  0.749279
               5   0.635514     0.630872  0.762795
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.654206     0.535986  0.833842
               5   0.598131     0.544863  0.824130
               10  0.588785     0.509364  0.839930
   equal-size  3   0.485981     0.907885  0.653384
               5   0.485981     0.907745  0.652530
               10  0.504673     0.823100  0.649869

In [8]:
tree_model = DecisionTree()
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df, min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


In [14]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 1.58 s.
Testing time (5, 'equal-width', 3): 0.11 s.
Training time (5, 'equal-width', 5): 1.33 s.
Testing time (5, 'equal-width', 5): 0.11 s.
Training time (5, 'equal-width', 10): 0.74 s.
Testing time (5, 'equal-width', 10): 0.11 s.
Training time (5, 'equal-size', 3): 1.60 s.
Testing time (5, 'equal-size', 3): 0.07 s.
Training time (5, 'equal-size', 5): 0.84 s.
Testing time (5, 'equal-size', 5): 0.07 s.
Training time (5, 'equal-size', 10): 0.44 s.
Testing time (5, 'equal-size', 10): 0.07 s.
Training time (10, 'equal-width', 3): 2.47 s.
Testing time (10, 'equal-width', 3): 0.06 s.
Training time (10, 'equal-width', 5): 1.56 s.
Testing time (10, 'equal-width', 5): 0.06 s.
Training time (10, 'equal-width', 10): 0.88 s.
Testing time (10, 'equal-width', 10): 0.06 s.
Training time (10, 'equal-size', 3): 1.51 s.
Testing time (10, 'equal-size', 3): 0.07 s.
Training time (10, 'equal-size', 5): 1.32 s.
Testing time (10, 'equal-size', 5): 0.07 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.635514     0.566861  0.794838
               5   0.644860     0.530257  0.812039
               10  0.579439     0.540913  0.802684
   equal-size  3   0.644860     0.678333  0.757026
               5   0.635514     0.644065  0.759834
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.654206     0.566439  0.816216
               5   0.626168     0.529316  0.832679
               10  0.598131     0.503810  0.845034
   equal-size  3   0.504673     0.905079  0.656123
               5   0.504673     0.902743  0.655598
               10  0.532710     0.868821  0.669287

In [15]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class DecisionForest

In [7]:
# Define the class DecisionForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# random_features: no. of features to evaluate at each split (default = 2), 0 means all features (no random sampling)
# notrees: no. of trees in the forest (default = 10)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a random forest (for details, see lecture slides)
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: Redefine divide_and_conquer to take one additional argument; random_features, and instead of
#         evaluating all features choose a random subset, e.g., by np.random.choice (without replacement)
# Hint 2: Generate each tree in the forest from a bootstrap replicate of df, e.g., by np.random.choice 
#         (with replacement) from the index values of df.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the mean of all relative class frequencies in the leaves of the forest into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(row), which for a test row
#         finds all leaf nodes and calculates the average of their class probabilities



In [9]:
class DecisionForest:
    
    def __init__(self):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.models = None

        
    def information_content(self, group_size, class_counts):
        group_weight = {k:group_size[k]/sum(group_size.values())  for k in group_size}
        InfRes = 0 
        for key in group_weight:
            tmp = {k:class_counts[k]  for k in class_counts if k[0] == key}
            tmp = {k:tmp[k]/sum(tmp.values())  for k in tmp}
            LnP = np.sum([-tmp[k]*np.log2(tmp[k]) for k in tmp])
            InfRes += group_weight[key]*LnP
        return InfRes        
    
    def divide_and_conquer(self, df, min_samples_split, nodeno, notree):

        best_idx = 0
        best_info = np.inf
        
        features = [x for x in df.columns if x != "CLASS"]
        for i in range(len(features)):
            ft = features[i]
            group_size = df[ft].value_counts().to_dict()  
            class_counts = df.groupby([ft, "CLASS"]).size().to_dict() 
            info = self.information_content(group_size, class_counts)
            if info < best_info:
                best_info = info
                best_idx = i
        best_feature = features[best_idx]


        possiblevalues = df[best_feature].unique()

        node_dict = {}
        df.set_index(best_feature, inplace=True)
        for pv in possiblevalues:
            
            self.max_nodes_no[notree] += 1
            node_dict[pv] = self.max_nodes_no[notree]
            
            new_df = df.loc[[pv]]

            if new_df.shape[0] <= min_samples_split or len(features) == 1 :
                self.models[notree].append((self.max_nodes_no[notree], "leaf", new_df.CLASS.value_counts(normalize=True)))
            else:
                self.models[notree].append((self.divide_and_conquer(new_df, min_samples_split, self.max_nodes_no[notree], notree)))
        res = (nodeno, best_feature, node_dict)
        return res

        
    def fit(self, df, nobins = 10, bintype = 'equal-width', min_samples_split = 5, random_features = 2, notrees = 10):
        self.notrees = notrees
        self.max_nodes_no = [0 for i in range(self.notrees)]
        self.models = [[] for i in range(self.notrees)]
        
        if random_features==0:
            random_features = df.shape[1]-2
        
        self.training_data = df.copy()
        self.training_labels = df["CLASS"].astype("category")
        self.labels = self.training_labels.cat.categories    
        self.count = self.training_labels.value_counts()  
        freq = self.count/sum(self.count)
        self.class_priors = freq.to_dict() # default class probabilities (relative frequencies of the class labels)
        
        del self.training_data["ID"]
        
        self.training_data, self.imputation = create_imputation(self.training_data)
        self.training_data, self.binning = create_bins(self.training_data, nobins = nobins, bintype = bintype)

        datalength = self.training_data.shape[0]
        featurelength = random_features
        
        features = self.training_data.columns
        features_idx = [i for i in range(len(features)) if features[i] != 'CLASS']
        label_loc = list(np.where(features == 'CLASS')[0])
        for i in range(notrees):
            data_idx = np.random.choice(datalength, size=datalength, replace=True)
            feat_idx = np.random.choice(features_idx, size=featurelength, replace=False)
            selected = self.training_data.iloc[data_idx, list(feat_idx)+(label_loc)]
            self.models[i].append(self.divide_and_conquer(selected, min_samples_split, 0, i))
            self.models[i] = sorted(self.models[i], key=lambda x:x[0])
                
 
    def make_prediction(self, nodeno, row, notree):
        nodeno, nodevalue, dic = self.models[notree][nodeno] 
        if nodevalue == "leaf":
            return dic.sort_index()
        else:
            row_feature_value = row[nodevalue]
            if row_feature_value in dic.keys():
                return self.make_prediction(dic[row_feature_value], row, notree) 
            else:
                # exception
                possiblenodes = list(dic.values())
                res = np.zeros([len(possiblenodes), len(self.labels)])
                for i in range(res.shape[0]):
                     res[i] = self.make_prediction(possiblenodes[i], row, notree) 
                res = np.mean(res, axis = 0)
                return res


    def predict(self, df):
        self.testing_data = df.copy()
  
        if "CLASS" in self.testing_data.columns:
            del self.testing_data["CLASS"]
        if "ID" in self.testing_data.columns:
            del self.testing_data["ID"]
        
        self.testing_data = apply_imputation(self.testing_data, self.imputation)
        self.testing_data = apply_bins(self.testing_data, self.binning)

        predictions = np.array([np.zeros(len(self.labels)) for i in range(self.testing_data.shape[0])])
        for i in range(self.testing_data.shape[0]):
            row = self.testing_data.iloc[i,:]
            
            pred = np.zeros([self.notrees, len(self.labels)])
            for j in range(self.notrees):
                pred[j] = self.make_prediction(0, row, j)
            predictions[i] = np.mean(pred, axis=0)

        predictions = pd.DataFrame(predictions, columns=self.labels)

        return predictions


In [10]:
# Test our code 

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labelsa = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 0.30 s.
Testing time (1, 1): 0.49 s.
Training time (1, 2): 1.04 s.
Testing time (1, 2): 0.63 s.
Training time (1, 5): 8.17 s.
Testing time (1, 5): 0.86 s.
Training time (2, 1): 0.29 s.
Testing time (2, 1): 0.47 s.
Training time (2, 2): 0.79 s.
Testing time (2, 2): 0.61 s.
Training time (2, 5): 6.03 s.
Testing time (2, 5): 1.25 s.
Training time (5, 1): 0.41 s.
Testing time (5, 1): 0.58 s.
Training time (5, 2): 1.13 s.
Testing time (5, 2): 0.82 s.
Training time (5, 5): 3.33 s.
Testing time (5, 5): 0.74 s.


Accuracy  Brier score       AUC
1 1  0.560748     0.615804  0.766736
  2  0.663551     0.562703  0.802252
  5  0.672897     0.475231  0.866712
2 1  0.542056     0.608380  0.746276
  2  0.635514     0.539378  0.828816
  5  0.635514     0.464603  0.875382
5 1  0.551402     0.627091  0.754944
  2  0.644860     0.531043  0.789631
  5  0.672897     0.443495  0.884956

In [11]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.79
AUC on training set: 0.96
Brier score on training set: 0.40


In [18]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labelsa = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 7.51 s.
Testing time (1, 1): 0.09 s.
Training time (1, 2): 10.65 s.
Testing time (1, 2): 0.09 s.
Training time (1, 5): 19.78 s.
Testing time (1, 5): 0.09 s.
Training time (2, 1): 7.97 s.
Testing time (2, 1): 0.09 s.
Training time (2, 2): 12.01 s.
Testing time (2, 2): 0.10 s.
Training time (2, 5): 19.42 s.
Testing time (2, 5): 0.10 s.
Training time (5, 1): 4.57 s.
Testing time (5, 1): 0.10 s.
Training time (5, 2): 7.31 s.
Testing time (5, 2): 0.09 s.
Training time (5, 5): 9.75 s.
Testing time (5, 5): 0.09 s.


Accuracy  Brier score       AUC
1 1  0.644860     0.477912  0.860736
  2  0.710280     0.411981  0.892842
  5  0.710280     0.421511  0.895716
2 1  0.663551     0.452421  0.872447
  2  0.626168     0.411684  0.911137
  5  0.654206     0.435847  0.881621
5 1  0.644860     0.457859  0.865636
  2  0.719626     0.410616  0.903016
  5  0.616822     0.455782  0.883897

In [24]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.96
AUC on training set: 1.00
Brier score on training set: 0.12


### Comment on assumptions, things that do not work properly, etc.